In [8]:
import random
import math
import os
import time
import gym
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical
from torch import FloatTensor, LongTensor
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

from ale_py.roms import SpaceInvaders
from ale_py import ALEInterface

In [9]:
DECAYRATE = 0.99
GAMMA = 0.99
LR = 0.0001
BATCHSIZE = 5

ale = ALEInterface()
ale.loadROM(SpaceInvaders)
env = gym.make('ALE/SpaceInvaders-v5', render_mode='human')
state = env.reset()
state_shape = state.shape

def imcrop(im):
    return im[20:-10]

net = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2),
    nn.Conv2d(in_channels=16, out_channels=48, kernel_size=3, stride=2),
    nn.Conv2d(in_channels=48, out_channels=64, kernel_size=3, stride=2),
    nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=2),
    nn.Flatten(),
    nn.Linear(2880, 512),
    nn.Linear(512, 96),
    nn.Linear(96, 6),
    nn.Sigmoid()
)

In [ ]:
episode = 76
net.load_state_dict(torch.load(os.path.join("/Users/stack/Documents/warudo/polgradchk", str(episode - 1) + ".pth")))
states, actions, rewards = [], [], []
steps = 0
optimizer = optim.RMSprop(net.parameters(), lr=LR)

while True:
    state = env.reset()
    
    # play out episode
    while True:
        probs = net(Variable(FloatTensor([imcrop(state)]).permute(0,3,1,2)))
        action = Categorical(probs).sample()
        action = action.data.numpy().astype(int)[0]
        next_state, reward, done, _ = env.step(action)
        if done:
            reward = 0
        states.append(Variable(FloatTensor([imcrop(state)]).permute(0,3,1,2)))
        actions.append(float(action))
        rewards.append(reward)
        state = next_state
        steps += 1
        if done:
            break    
    
    # update policy
    if episode % BATCHSIZE == 0:
        print("updating policy")
        # strange implementation of discounted reward
        running_add = 0
        for i in reversed(range(steps)):
            if rewards[i] == 0:
                running_add = 0
            else:
                running_add = GAMMA * running_add + rewards[i]
                rewards[i] = running_add
        
        # normalize reward
        m, s = np.mean(rewards), np.std(rewards)
        for i in range(steps):
            rewards[i] = (rewards[i] - m) / s
        
        optimizer.zero_grad()
        for i in range(steps):
            state = states[i]
            action = Variable(FloatTensor([actions[i]]))
            reward = rewards[i]
            probs = net(state)
            loss = -Categorical(probs).log_prob(action) * reward
            loss.backward()
        optimizer.step()
        
        torch.save(net.state_dict(), os.path.join("/Users/stack/Documents/warudo/polgradchk", str(episode) + ".pth"))
        
        states, actions, rewards = [], [], []
        steps = 0
    
    print("episode " + str(episode) + " done")
    episode += 1